In [4]:
import jaxley as jx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
import glob

# ---------------------------------------------------------
# 1. CONFIGURATION
# ---------------------------------------------------------
# TYPE_NAME = "CT1"
# # glob for the body_id
# swcs_with_type = glob.glob(f"./morphologies/{TYPE_NAME}_*.swc")
# body_id = swcs_with_type[0].split(f"{TYPE_NAME}_")[1].split("_")[0]
BODY_ID = 720575940626979621

SWC_PATH = "./morphologies/fafb_fafb_fafb_space_swc_720575940626979621_CT1.swc"

In [ ]:
# ---------------------------------------------------------
# 2. LOAD AND SCALE MORPHOLOGY
# ---------------------------------------------------------
print(f"Loading SWC: {SWC_PATH}")
# Jaxley reads SWC. Note: We assume the SWC is in nanometers (NeuPrint default).
ct1 = jx.read_swc(SWC_PATH, ncomp=1, ) # n_comp=1 means 1 compartment per SWC segment

Loading SWC: ./morphologies/fafb_fafb_fafb_space_swc_720575940626979621_CT1.swc


/groups/turaga/home/ellisj1/connectome_compartments/compartment_env/lib/python3.12/site-packages/jaxley/utils/morph_attributes.py:244: RuntimeWarning: divide by zero encountered in divide
  lengths[is_constant_radius] / radius_start[is_constant_radius] ** 2


AssertionError: Radius 0.0 in SWC file. Set `read_swc(..., min_radius=...)`.

In [35]:
synapses = pd.read_feather("morphologies/fafb_783_synapses.feather")

In [74]:
ct1.nodes

,local_cell_index,local_branch_index,local_comp_index,length,radius,axial_resistivity,capacitance,v,area,volume,...,x,y,z,global_cell_index,global_branch_index,global_comp_index,controlled_by_param,apical,basal,custom7
0,0,0,0,7095.073563,846.348300,5000.0,1.0,-70.0,3.829834e+07,1.858249e+10,...,296063.750000,303028.78125,128936.648438,0,0,0,0,False,False,True
1,0,1,0,2969.860305,160.851312,5000.0,1.0,-70.0,3.003156e+06,2.422283e+08,...,294106.062500,305466.34375,132744.656250,0,1,1,0,True,False,False
2,0,2,0,1312.386060,45.855334,5000.0,1.0,-70.0,3.781227e+05,8.670876e+06,...,293380.937500,306895.78125,132915.859375,0,2,2,0,False,True,False
3,0,3,0,8929.148443,127.167298,5000.0,1.0,-70.0,7.147722e+06,4.708175e+08,...,294633.531250,310590.65625,134663.812500,0,3,3,0,True,False,False
4,0,4,0,21783.638837,115.036335,5000.0,1.0,-70.0,1.576097e+07,9.416213e+08,...,292114.781250,320539.87500,139675.515625,0,4,4,0,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,0,1080,0,398.460895,89.750000,5000.0,1.0,-70.0,2.364424e+05,1.185988e+07,...,260500.031250,360415.59375,209744.593750,0,1080,1080,0,False,True,False
1081,0,1081,0,182.853908,105.500000,5000.0,1.0,-70.0,1.850796e+05,8.525067e+06,...,260493.406250,360558.00000,208595.625000,0,1081,1081,0,False,True,False
1082,0,1082,0,998.117961,95.479629,5000.0,1.0,-70.0,6.022299e+05,2.976829e+07,...,260123.046875,358806.06250,205001.578125,0,1082,1082,0,False,True,False
1083,0,1083,0,1080.096353,85.314093,5000.0,1.0,-70.0,5.875330e+05,3.274579e+07,...,276080.500000,346475.59375,168812.156250,0,1083,1083,0,False,True,False


In [75]:
296063.750000/4,	303028.78125/4,	128936.648438/40

(74015.9375, 75757.1953125, 3223.4162109500003)

In [37]:
synapses.head()

,id,connector_id,x,y,z,confidence,syn_top_p,syn_top_nt,gaba,acetylcholine,...,octopamine,serotonin,dopamine,prepost,pre,post,neuropil,label,post_label,side
0,30792687,9458798388,701772,263419,205120,162,0.94,acetylcholine,0.01,0.94,...,0.01,0.00,0.03,0,720575940596125868,720575940627502824,LO,unknown,dendrite,left
1,30792740,9458798388,700360,263564,205360,119,0.77,acetylcholine,0.03,0.77,...,0.01,0.01,0.09,0,720575940596125868,720575940630993423,LO,unknown,unknown,left
2,30795760,9461355948,705594,264618,206280,170,0.84,acetylcholine,0.01,0.84,...,0.08,0.00,0.00,0,720575940596125868,720575940618477947,LO,unknown,primary.dendrite,left
3,30796555,9591223614,698424,265904,205840,131,0.84,acetylcholine,0.02,0.84,...,0.05,0.00,0.03,0,720575940596125868,720575940621780812,LO,unknown,dendrite,left
4,32020018,9458798388,701740,263350,205080,136,0.74,acetylcholine,0.03,0.74,...,0.01,0.01,0.12,0,720575940596125868,720575940622955561,LO,unknown,unknown,left


In [41]:
synapses_prepost = synapses[(synapses.pre == "720575940628908548")]# | (synapses.post == BODY_ID)]
synapses_prepost.shape

(0, 21)

In [ ]:
# flywire.set_chunkedgraph_secret("380570890ba8442e28f0f0c7f388bfb5")

Token succesfully stored.


In [6]:
from fafbseg import flywire

synapses = flywire.get_synapses(
    720575940626979621,
    pre=True,
    post=True,
    transmitters=True, 
    neuropils=True, 
    materialization="auto"
)

/groups/turaga/home/ellisj1/connectome_compartments/compartment_env/lib/python3.12/site-packages/blessed/terminal.py:222: UserWarning: Failed to setupterm(kind='xterm-color'): setupterm: could not find terminfo database
  warnings.warn(msg)


Using materialization version 783.


In [20]:
skeleton = flywire.get_skeletons(BODY_ID, dataset=783)
skeleton.nodes[['x', 'y', 'z', 'radius']] *= 0.001

In [21]:
skeleton.nodes

,node_id,x,y,z,parent_id,radius,type
0,0,783.222412,251.957306,199.344757,156892,0.175,slab
1,1,500.139801,194.059418,49.563923,11,0.059,end
2,2,742.761414,184.378998,190.385452,101076,0.146,slab
3,3,732.077148,286.954742,175.604874,77880,0.000,end
4,4,730.419006,307.341309,147.597305,75009,0.172,end
...,...,...,...,...,...,...,...
165593,165593,801.449341,257.894531,230.436844,165580,0.093,slab
165594,165594,801.596558,271.967316,230.201019,165589,0.140,slab
165595,165595,801.488708,272.857819,230.043045,165596,0.101,end
165596,165596,801.458191,272.318726,230.182571,165594,0.173,slab


In [17]:
skeleton.to_swc(f"./morphologies/fafb_fafb_fafb_space_swc_{BODY_ID}_CT1.swc")
ct1 = jx.read_swc(f"./morphologies/fafb_fafb_fafb_space_swc_{BODY_ID}_CT1.swc", ncomp=1, min_radius=.01)

/groups/turaga/home/ellisj1/connectome_compartments/compartment_env/lib/python3.12/site-packages/jaxley/utils/morph_attributes.py:244: RuntimeWarning: divide by zero encountered in divide
  lengths[is_constant_radius] / radius_start[is_constant_radius] ** 2
/groups/turaga/home/ellisj1/connectome_compartments/compartment_env/lib/python3.12/site-packages/jaxley/utils/morph_attributes.py:252: RuntimeWarning: divide by zero encountered in divide
  * (1 / radius_start[is_varying_radius] - 1 / radius_end[is_varying_radius])
/groups/turaga/home/ellisj1/connectome_compartments/compartment_env/lib/python3.12/site-packages/jaxley/utils/morph_attributes.py:244: RuntimeWarning: divide by zero encountered in divide
  lengths[is_constant_radius] / radius_start[is_constant_radius] ** 2
/groups/turaga/home/ellisj1/connectome_compartments/compartment_env/lib/python3.12/site-packages/jaxley/utils/morph_attributes.py:252: RuntimeWarning: divide by zero encountered in divide
  * (1 / radius_start[is_varyi

KeyboardInterrupt: 

In [67]:
input_coords = synapses[['pre_x', 'pre_y', 'pre_z']].to_numpy()
all_comp_indices = ct1.nodes.index.to_numpy()
tree = KDTree(ct1.nodes[['x', 'y', 'z']].values)
dists_in, idxs_in = tree.query(input_coords) 
synapses['jaxley_branch'] = [all_comp_indices[i] for i in idxs_in]

In [68]:
input_coords

array([[770176, 301152, 181520],
       [725788, 243572, 199760],
       [693944, 289312, 198840],
       ...,
       [743656, 233964, 185720],
       [747400, 233232, 170920],
       [744052, 264632, 179600]], shape=(214717, 3), dtype=object)

In [63]:
ct1.nodes[["x", "y", "z"]].values

array([[296063.75     , 303028.78125  , 128936.6484375],
       [294106.0625   , 305466.34375  , 132744.65625  ],
       [293380.9375   , 306895.78125  , 132915.859375 ],
       ...,
       [260123.046875 , 358806.0625   , 205001.578125 ],
       [276080.5      , 346475.59375  , 168812.15625  ],
       [285474.125    , 324243.9375   , 144926.09375  ]], shape=(1085, 3))

In [73]:
np.linalg.norm(ct1.nodes[["x", "y", "z"]].values - np.array([770176, 301152, 181520]), axis=1).min()

np.float64(477023.01492639567)

In [58]:
synapses

,pre,post,cleft_score,pre_x,pre_y,pre_z,post_x,post_y,post_z,id,gaba,acetylcholine,glutamate,octopamine,serotonin,dopamine,neuropil,jaxley_branch
0,720575940635282332,720575940626979621,126.0,770176,301152,181520,770124,301304,181480,156524963,0.035845,0.85906,0.033672,0.020023,0.002569,0.04883,ME_R,1051
1,720575940626434348,720575940626979621,133.0,725788,243572,199760,725840,243688,199760,1161298,0.006436,0.967498,0.004244,0.017165,0.000051,0.004607,LO_R,1051
2,720575940630130176,720575940626979621,105.0,693944,289312,198840,693984,289428,198800,1387239,0.390243,0.471311,0.038795,0.098353,0.00004,0.001259,LO_R,1051
3,720575940604872416,720575940626979621,120.0,681200,333632,166040,681200,333760,166000,159101474,0.617959,0.355747,0.023302,0.001558,0.000015,0.00142,LO_R,1051
4,720575940611837304,720575940626979621,165.0,716364,328980,154640,716332,328868,154600,142842206,0.025091,0.948199,0.003141,0.023226,0.000038,0.000305,ME_R,1051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214712,720575940626979621,720575940626638384,147.0,733116,196360,194240,733272,196304,194280,195514476,0.998948,0.001008,0.000034,0.000005,0.0,0.000005,LO_R,1051
214713,720575940626979621,720575940627160143,164.0,763356,209320,194000,763328,209204,194000,196779482,0.642752,0.071584,0.219868,0.02691,0.005637,0.033248,ME_R,1051
214714,720575940626979621,720575940621597445,103.0,743656,233964,185720,743728,233996,185720,183917990,0.832996,0.0648,0.056879,0.011956,0.005534,0.027835,LO_R,1051
214715,720575940626979621,720575940638088026,145.0,747400,233232,170920,747232,233312,170880,161296976,0.99683,0.000407,0.002755,0.000003,0.0,0.000005,LO_R,1051
